In [1]:
import pandas as pd
import numpy as np

In [2]:
DB_PATH = "Simulation/FSN_Data_5k.db"

In [3]:
from NetEmbs.utils.IO.db_connection import upload_data, upload_journal_entries

In [5]:
upload_data()

TypeError: upload_data() missing 1 required positional argument: 'path'